In [ ]:
!pwd

In [1]:
import torch
import cvxpy as cp
import pickle
import numpy as np
import os

def load_gradients(file_name):
    with open(file_name,'rb') as f:
        hm = pickle.load(f)

    L = hm['Ltilde'].shape[0]
    index2layerscheme = [None for i in range(L)]

    for name in hm['layer_index']:
        index = hm['layer_index'][name]
        layer_name = name[:-11]
        scheme = name[-11:]
        a = hm['Ltilde']
        #print(f'index {index} layer {layer_name} scheme {scheme} Ltilde {a[index,index].item():.6f}')
        index2layerscheme[index] = (layer_name,scheme)
    
    cached_grad = np.zeros_like(hm['Ltilde'])
    for i in range(L):
        for j in range(L):
            layer_i,scheme_i = index2layerscheme[i]
            layer_j,scheme_j = index2layerscheme[j]
            if layer_i == layer_j:
                if scheme_i == scheme_j:
                    cached_grad[i,j] = cached_grad[j,i] = 2 * hm['Ltilde'][i,j]
                else:
                    cached_grad[i,j] = cached_grad[j,i] = 0
            else:
                cached_grad[i,j] = cached_grad[j,i] = hm['Ltilde'][i,j] - hm['Ltilde'][i,i] - hm['Ltilde'][j,j]

    return hm, cached_grad, index2layerscheme

In [2]:
batch_Ltildes_clado = []
batch_Ltildes_clado2 = []
calib_num = 128
for b in range(1,2):
    
#     with open(f'./clado_mpqco_results/bert-base/Ltilde_bert-base/Ltilde_calib{calib_num}_batch{b}(size8).pkl','rb') as f:
#         hm = pickle.load(f)
#         batch_Ltildes_clado.append(hm['Ltilde'])
    hm,cached_grad,_ =  load_gradients(f'./clado_mpqco_results/bert-base/Ltilde_bert-base/Ltilde_calib{calib_num}_batch{b}(size8).pkl')  

    batch_Ltildes_clado2.append(hm['Ltilde'])

In [4]:
len(batch_Ltildes_clado2)

1

In [5]:
#batch_Ltildes_clado = np.array(batch_Ltildes_clado).mean(axis=0)
batch_Ltildes_clado2 = np.array(batch_Ltildes_clado2).mean(axis=0)

In [ ]:
batch_Ltildes_clado2.shape

In [8]:
x = np.diag(batch_Ltildes_clado2)
(np.argmin(x.reshape(-1,3),axis=1)).reshape(-1,6)
# q k v AttentionOut intDense OutDense 

array([[0, 1, 2, 0, 2, 0],
       [2, 2, 2, 0, 1, 2],
       [1, 2, 1, 1, 1, 0],
       [1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [1, 0, 0, 1, 1, 0],
       [1, 0, 0, 1, 1, 1],
       [2, 1, 2, 2, 0, 0],
       [0, 1, 2, 2, 1, 0],
       [1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 1]])

In [7]:
y = []
for item in ((np.argmin(x.reshape(-1,3),axis=1))):
    if item == 2:
        y += [0,0,1]
    elif item == 1:
        y += [0,1,0]
    else:
        y += [1,0,0]
print(y)

[1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0]


In [ ]:
y = []
for item in range(72):
    y += [1,0,0]
print(y)

In [ ]:
u,s,v = np.linalg.svd(batch_Ltildes_clado2)
s[s<0] = 0
psd_ltilde = u@np.diagonal(s)@v
(np.argmin(psd_ltilde.reshape(-1,3),axis=1))

In [ ]:
hm['layer_index']